<a href="https://colab.research.google.com/github/painterV/some_coding/blob/main/lab07_ohie_probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OHIE

The Oregon Health Insurance Experiment (OHIE) was a program run in
Oregon, USA in 2008 in which certain residents of that state were
offered the opportunity to enroll in a subsidized health insurance
program.  To allocate this opportunity fairly, interested people
were invited to participate in a lottery.  The people who won the
lottery ("treated") were then given the opportunity to apply to the insurance
program.  A subset of these people actually applied to the program,
and finally a subset of these applicants who were confirmed to be
eligible were granted insurance.

Since the opportunity to apply for insurance was allocated randomly with all households having equal probability of being selected,
this program is essentially a randomized experiment (although the
randomization was employed for fairness, not to facilitate
research).  In particular, there was great interest in the outcomes
over the subsequent several years of the people who were awarded
insurance, compared to those who participated in the lottery but
were not selected.  Since this selection was random, in principal
both the treatment measurment should not be associated with any other measurement. For example, treated households should be similar in age or income to the non-treated households.

In this notebook, we only consider the "baseline" information,
namely, characteristics of the individuals who applied to the
lottery.  We also know who "won" the lottery, who among those given
the opportunity to apply for insurance actually did so, and who
among those who applied for insurance were deemed to be eligible and
granted insurance.

A primary focus of this notebook is to use the OHIE data to
illustrate concepts from probability, including conditional
probabilities and conditional independence.

In [ ]:
import os
import pandas as pd
import numpy as np

Load the OHIE data from a file:

In [ ]:
section = "100"
base = "/scratch/stats206s%sf22_class_root/stats206s%sf22_class/materials/data" % (section, section)
df = pd.read_csv(os.path.join(base, "oregonhie.csv.gz"))
df.columns

## Proportions

Before beginning, subsequent computations are little easier if we translate the way the data were originally coded into True/False values instead of the two level categories they are stored as originally.

In [ ]:
df["female_list"].value_counts(dropna = False)

In [ ]:
df["female"] = df["female_list"] == "1: Female"

df["have_phone_list"].value_counts(dropna = False)

In [ ]:
df["has_phone"] = df["have_phone_list"] == "Gave Phone Number"

df["treatment"].value_counts(dropna = False)

In [ ]:
df["treatment"] = df["treatment"] == "Selected"

df["applied_app"].value_counts(dropna = False)

In [ ]:
df["applied_app"] = df["applied_app"] == "Submitted an Application to OHP"

df["approved_app"].value_counts(dropna = False)

In [ ]:
df["approved_app"] = df["approved_app"] == "Yes"

df["zip_msa_list"].value_counts(dropna = False)

In [ ]:
df["zip_msa"] = df["zip_msa_list"] == "Zip code of residence in a MSA"

Why did we transform these labels to True/False values? To remind yourself, compute the proportion of people in the table that are female. What proportion are both female and were selected for treatment?

<details>
    
```    
(df["female"].mean(), (df["female"] & df["treatment"]).mean())
```
    
</details>

Suppose we selected a random person from this table. What is the probability that we would select a female? What is the probability we would select someone who is both female and treated?

Recall the definition of probability of an event:

$$P(\text{event}) = \frac{\text{all outcomes where event occurs}}{\text{all possible outcomes}}$$

For the event "is female", what are the numerator and denominator in this equation?

<details>
```
[df["female"].sum(), # over
 df.shape[0]]
```
</details>

Let's verify the idea of proportions as **empirical probabilities** (probability of event in occuring in data set). Use the [sample method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) to randomly select 1 person 1000000 times using `n = 1000000` and `replace = True`. What proportion of these people were female? Female and treated? (The numbers won't be exactly the same, but they should be very close.)

<details>
```
sampled_people = df.sample(n = 1000000, replace = True)
[sampled_people["female"].mean(),
    (sampled_people["female"] & sampled_people["treatment"]).mean()]
```
</details>

## Households

One particular feature of this lottery in Oregon is that treatment was allocated by household, not by individuals. Some of the probabilities we will want to calculate must be done so at the household level.

To confirm that treatment was selected at the household level, group by `"household_id"` and run the `.nunique()` method on the `"treatment"` column. Are any of the values greater than 1 (indicating the members of the same household receieved different treatment outcomes)?

<details>
    
```
(df.groupby("household_id")["treatment"].nunique() > 1).any()
```
    
</details>


Create a table `hh` that aggregates to the household level and uses the `first` method to for the 'treatment' and 'zip_msa' columns. For 'applied_app' and 'approved_app' use `max` (which will get the value `True` if anyone in the household applied or was approved). For the columns "birthyear_list" use `median`, and for 'female' and 'has_phone' use `mean`.

<details>

```
hh = df.groupby('household_id').agg({'treatment': 'first',
                                     'applied_app': 'max',
                                     'approved_app': 'max',
                                     'zip_msa': 'first',
                                     'birthyear_list': 'median',
                                     'female': 'mean',
                                     'has_phone': 'mean'
                                    })
hh
```

</details>


Let's begin by looking at how many households fall into each category of
program participation. Group the `hh` table by "treatment", "applied_app", and "approved_app" and use the `size()` method to see how many people fall into each of the four status categories: not
selected, selected but did not apply, applied but deemed ineligible,
applied and deemed eligible.

<details>

```
status = hh.groupby(["treatment", "applied_app", "approved_app"]).size()
status
```

</details>



It is desirable to see these counts presented as proportions.  Convert your counts to proportion by dividing by the total.

<details>

```
status_prop = status / status.sum()
status_prop
```

</details>



Verify that all these proportions sum to 1, as they must do by the fact that the probability that some event happens must be one (the law of total probablity)

<details>

```
status_prop.sum()
```

</details>


## Probabilities of events and complements

Tables of proportions can be useful for determining probabilities of many combinations of outcomes. What is the probability that a randomly selected household would win the lottery (treatment), but fail to apply for coverage?

<details>

```
status_prop[True, False, False] # or status_prop[True, False,
```

</details>


Another way to think about probability for events that includes multiple outcomes is that it is the sum of probabilities of all individual outcomes where the event occurs. What is the probability that a household applies for treatment and also has a successful application? Verify this use the previous results and using the `hh` table. (Due to how missing values are handled, there may be very small discrepancies in these numbers.)

<details>

```
status_prop[True, True].sum()
```

</details>


<details>

```
(hh["treatment"] & hh["applied_app"]).mean()
```

</details>


Using either the proportions table we have been working on or the `hh` table, what is the probability of a household being selected for treatment? We call this a **marginal probability** (marginal here means "all by itself" as compared to a "joint probability" involving two or more outcomes).

<details>

```
status_prop[True].sum()
```

</details>


<details>

```
hh["treatment"].mean()

```

</details>

The 'complementary event' to being selected is to not be selected.
That is, everyone involved is either selected or not selected, and
nobody can be both.  Therefore, another way to calculate the
probability of being selected is to take 1 minus the probability of
not being selected. Use the complement with the proportions table of not being selected to get the probability of being selected.

<details>

```
1 - status_prop[False]

```

</details>

The state of Oregon had a fixed budget for this program, and could
not know in advance how many of the people who were selected in the
lottery would be deemed eligible for insurance.  Therefore, the
probability of being selected in the lottery was presumably set to a
conservative level.

# Joint probabilities

Events (including several we have seen already) can involve more than one outcome. For example, we have already computed the probability that a household won the lottery but failed to receive apply to the program.

Let's now turn to some joint probabilities of the available demographic information along with treatment status for the household.

We will start using an age group variable that we construct below.
First we obtain the age of each subject in the first year of the
program, then ask Pandas to group the subjects into three groups of
equal size based on age.

In [ ]:
hh = df.groupby('household_id').agg({'treatment': 'first',
                                     'applied_app': 'max',
                                     'approved_app': 'max',
                                     'zip_msa': 'first',
                                     'birthyear_list': 'median',
                                     'female': 'mean',
                                     'has_phone': 'mean'
                                    })
hh["age"] = 2008 - hh["birthyear_list"]
hh["agegrp"] = pd.qcut(hh["age"], 3)

Construct a 'contingency table' or 'crosstab' showing how many people have
each combination of the age group and treatment variables. (Hint: use `groupby`, `size`, and `unstack`)

<details>

```
counts = hh.groupby(["agegrp", "treatment"]).size().unstack()
counts
```

</details>


Convert the counts above to proportions. (Hint: you may have to use `sum` twice)

<details>

```
probs = counts / counts.sum().sum()
probs
```

</details>


The previous table contains sample proportions, which must sum to exactly 1. Show that the values sum to 1.

<details>

```
probs.sum().sum()
```

</details>


Find the value of

$$P(\text{Won the lottery and in the old age category})$$

(Hint: you may need to refer to your row using position rather than name)

<details>

```
probs[True][2]
```

</details>


Repeat these computations to find the joint probabilities for treatment and whether the household lives in a metropolitan area (`"zip_msa"`). What was the most common joint outcome. Describe this in words.

<details>

```
count_msa = hh.groupby(["zip_msa", "treatment"]).size().unstack()
probs_msa = count_msa / count_msa.sum().sum()
probs_msa
```

</details>
We see that most common outcome was households in metropolitan areas not winning the lottery.